### RandomForest Feature Importance for Selection

Process steps:
* Build full dataset with decoys and compounds
* Build compound-only dataset
* Preprocess categorical with one-hot-encoding
* Find intersecting columns and filter compound-only to these
* Add non-linear transformations to compound-only dataset
* Create binary target with IC50 <= 10 as 1, else 0
* Using compound-only df, fine-tune random forest with GridSearchCV (CV = count of positive class)
* Extract feature importance and remove any with importance of 0
* Create important features in full dataset

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import skewtest
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import Imputer
from sklearn.metrics import confusion_matrix, log_loss
from custom_functions import *

In [2]:
avail_transformations = ["log", "log2", "log10", "cubert", 
                         "sqrt", "exp", "exp2", "cube", "sq"]

* Build full-dataset
* Build compound-dataset
* Preprocess data adding one-hot-encoded features for both
* Find intersecting columns
* Add non-linear transformations and drop na's

In [3]:
# Load in full dataset
full_x, full_y = load_full_dataset()
# Preprocess variables
full_x = preprocess_variables(full_x)
# Extract list of available columns
full_columns = full_x.columns
print("Loading in compound dataset....")
# Read in compound dataset
compound_x, compound_y = load_compound_dataset()
# Preprocess
compound_x = preprocess_variables(compound_x)
# Find intersecting features
avail_columns = compound_x.columns.intersection(full_columns)
# Select features on subset
x_data = compound_x.loc[:, avail_columns]
y_data = compound_y.copy()
print("Adding non-linear features to compound dataset....")
# Add all transformations on compound data
for feature in x_data.columns[x_data.dtypes == 'float64']:
    x_data = add_transformations(x_data, feature)
# Drop any new columns with NaN due to improper transformation
x_data.replace([np.inf, -np.inf], np.nan, inplace=True)
x_data.dropna(axis=1, inplace=True)
assert not sum(x_data.isna().sum()), "Unexpected nulls found"

Adding Akt1_decoys_padel.csv....
Adding AmpC_decoys_padel.csv....
Adding cp3a4_decoys_padel.csv....
Adding cxcr4_decoys_padel.csv....
Adding gcr_decoys_padel.csv....
Adding HIVpr_decoys_padel.csv....
Adding HIVrt_decoys_padel.csv....
Adding Kif11_decoys_padel.csv....
Loading in compound dataset....
Adding non-linear features to compound dataset....


In [4]:
# Create binary variable
y_class = np.squeeze([int(y_val <= 10) for y_val in y_data])

RandomForest

In [10]:
# How well does random forest predict potency?
print("Tuning random forest on compound dataset....")
rf_model = RandomForestClassifier(random_state=0, 
                               class_weight={0: 1-np.mean(y_class), 
                                             1: np.mean(y_class)})
params = {"n_estimators": [30, 40, 50],
          "max_depth": [3, 4, 5],
          "min_samples_split": [2, 3],
         "criterion": ["entropy"],
         "max_features": ["auto"]}
grid = GridSearchCV(estimator=rf_model, param_grid=params, cv=5, n_jobs=3)

grid.fit(x_data, y_class)
print(grid.best_params_)
best_rf_model = grid.best_estimator_

Tuning random forest on compound dataset....
{'max_features': 'auto', 'min_samples_split': 2, 'criterion': 'entropy', 'max_depth': 4, 'n_estimators': 40}


In [12]:
# Analyze CV prediction performance
predict = cross_val_predict(
    best_rf_model, x_data, y_class, cv=sum(y_class), method="predict")

predict_proba = cross_val_predict(
    best_rf_model, x_data, y_class, cv=sum(y_class), method="predict_proba")

print(confusion_matrix(y_class, predict, labels=[1, 0]))
print(np.array([["TP", "FN"], ["FP", "TN"]]))

[[ 3  8]
 [ 4 32]]
[['TP' 'FN']
 ['FP' 'TN']]


AdaBoostClassifier

In [21]:
from sklearn.ensemble import AdaBoostClassifier
# How well does AdaBoost predict potency?
print("Tuning AdaBoost on compound dataset....")
model = AdaBoostClassifier(random_state=0)
params = {"n_estimators": [35, 40, 45],
          "learning_rate": [0.05, 0.075]}
grid = GridSearchCV(estimator=model, param_grid=params, cv=5, n_jobs=3)

grid.fit(x_data, y_class)
print(grid.best_params_)
best_model = grid.best_estimator_

Tuning AdaBoost on compound dataset....
{'n_estimators': 40, 'learning_rate': 0.05}


In [22]:
# Analyze CV prediction performance
predict = cross_val_predict(
    best_model, x_data, y_class, cv=sum(y_class), method="predict")

predict_proba = cross_val_predict(
    best_model, x_data, y_class, cv=sum(y_class), method="predict_proba")

print(confusion_matrix(y_class, predict, labels=[1, 0]))
print(np.array([["TP", "FN"], ["FP", "TN"]]))


[[ 6  5]
 [ 5 31]]
[['TP' 'FN']
 ['FP' 'TN']]


In [23]:
# Analyze CV Performance
pd.DataFrame(
    {"IC50": y_data, 
     "y_class": y_class, 
     "Prediction": predict, 
     "Proba": predict_proba[:,1]})[
    ["IC50", "y_class", "Prediction", "Proba"]]

,IC50,y_class,Prediction,Proba
0,0.036,1,1,0.999833
1,10.000,1,0,0.020414
2,50.000,0,0,0.012971
3,50.000,0,1,0.938043
4,50.000,0,0,0.012011
5,8.000,1,1,0.999862
6,50.000,0,0,0.012011
7,50.000,0,0,0.020414
8,35.000,0,0,0.020414
9,50.000,0,0,0.024974


In [24]:
# Analyze train prediction performance
best_model.fit(x_data, y_class)
predict = best_model.predict(x_data)
print(confusion_matrix(y_class, predict, labels=[1, 0]))
print(np.array([["TP", "FN"], ["FP", "TN"]]))

[[11  0]
 [ 0 36]]
[['TP' 'FN']
 ['FP' 'TN']]


In [25]:
# Which ones did it get without CV?
pd.DataFrame({"IC50": y_data, "y_class": 
              y_class, "Prediction": predict})

,IC50,Prediction,y_class
0,0.036,1,1
1,10.000,1,1
2,50.000,0,0
3,50.000,0,0
4,50.000,0,0
5,8.000,1,1
6,50.000,0,0
7,50.000,0,0
8,35.000,0,0
9,50.000,0,0


Analyze feature importance

In [26]:
best_model.fit(x_data, y_class)
feat_importance = best_model.feature_importances_
best_features = [
    (f, i) for i, f in sorted(zip(feat_importance, x_data.columns), 
                              reverse=True) if i != 0]
feat_df = pd.DataFrame(best_features)
feat_df.columns = ["Feature", "Importance"]
feat_df

,Feature,Importance
0,MATS5p_sq,0.175
1,MATS5m,0.150
2,ATSC6v,0.125
3,ATSC6v_sq,0.100
4,MATS5p_cube,0.075
5,MATS5m_exp2,0.075
6,MATS5m_exp,0.075
7,MATS5m_cube,0.075
8,MATS5p_exp2,0.025
9,MATS5p_exp,0.025


**Analysis:**

When using LOOCV (leaving one positive class out each split), 3 relatively-potent compounds are identified. This shows promise that at least some of the characteristics associated with potency are discovered and leveraged for prediction.

The more potent compounds, however, are not identified through the prediction when using cross-validation. This would indicate that the primary characteristics of potency discovered thus-far are not leveraged in this predictive model.

When analyzing the train error on the compound dataset, the model overfits and accurately predicts the potency class of every compound. The information this can provide us with is what features are used to make this distinction. We should try to leverage the features identified as most important for further exploration and use as a basis for the data augmentation analysis when analyzing the impact of having the decoys added to the dataset.

We can also try to leverage these same features (144 important features) in different models to see if they are able to find better seperation between potent/non-potent when using LOOCV (one split per positive example).